**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning - Ahmed Skander KARKAR

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer__: When ```train``` is ```True```, the functin ```act``` explores (i.e. chooses an action at random) with probability ```epsilon```. With probability 1-```epsilon```, the function chooses the best action in the current state according to the current learned policy. This is called exploitation (of the learned policy). ```epsilon``` is essential as is it calibrates exploration and exploitation, both essential to learning. When ```train``` is false, we want to stop exploration of new actions and only follow the learned policy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win - lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board > 0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 # I changed this line. It was exactly the same as the line above it.

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 # I changed this line. It was exactly the same as the line above it.
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 100 # set small when debugging
epochs_test = 20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Answer__: ```position``` and ```board``` represent the current state of the environment. ```board``` contains the positions of the cheese cells and the poisonous cells. ```position``` indicates the rat's position on the board and where it can go.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0] # random action regardless of the state
        return(a)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE; I used the code from above
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = False) # testing phase, no more training

            # Apply an action to the environment, get the next state, the reward
            # and if the game ends
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,5,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/18.0. Average score (-8.0)
Win/lose count 10.0/9.0. Average score (-3.5)
Win/lose count 12.0/11.0. Average score (-2.0)
Win/lose count 14.0/22.0. Average score (-3.5)
Win/lose count 8.5/10.0. Average score (-3.1)
Final score: -3.1


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer__: To prove the first equality, we decompose the expected value by the possible next state and action
\begin{align*}
\mathcal{Q}^{\pi}(s,a) &= \mathbb{E}_{p^{\pi}} [ \sum_{t \geq 0} \gamma^t r(s_t, a_t) | s_0 = s, a_0 = a]   \\
&= \mathbb{E}_{p^{\pi}} [r(s_0, a_0) + \sum_{t \geq 1} \gamma^t r(s_t, a_t) | s_0 = s, a_0 = a] \\
&= r(s, a) + \mathbb{E}_{p^{\pi}} [ \sum_{t \geq 1} \gamma^t r(s_t, a_t) | s_0 = s, a_0 = a] \\
&= r(s, a) + \sum_{(s', a')} \mathbb{E}_{p^{\pi}} [\sum_{t \geq 1} \gamma^t r(s_t, a_t) \mathbb{1}_{s_1 = s', a_1 = a'} | s_0 = s, a_0 = a] \\
&= r(s, a) + \sum_{(s', a')} \mathbb{E}_{p^{\pi}} [ \mathbb{E}_{p^{\pi}} (\sum_{t \geq 1} \gamma^t r(s_t, a_t) \mathbb{1}_{s_1 = s', a_1 = a'} | s_0 = s, a_0 = a, s_1 = s', a_1 = a') | s_0 = s, a_0 = a] \\
&= r(s,a) + \sum_{(s', a')} \mathbb{E}_{p^{\pi}} [ \mathbb{1}_{s_1 = s', a_1 = a'} \mathbb{E}_{p^{\pi}} (\sum_{t \geq 1} \gamma^t r(s_t, a_t) | s_1 = s', a_1 = a') | s_0 = s, a_0 = a] \\
&= r(s, a) \gamma \sum_{(s', a')} \mathbb{E}_{p^{\pi}} [ \mathbb{1}_{s_1 = s', a_1 = a'} \mathbb{E}_{p^{\pi}} (\sum_{t \geq 1} \gamma^{t-1} r(s_t, a_t) | s_1 = s', a_1 = a') | s_0 = s, a_0 = a] \\
&= r(s, a) + \gamma \sum_{(s', a')} p^{\pi}(s', a' | s, a) \mathcal{Q}^{\pi}(s', a') \\
&= \mathbb{E}_{(s', a') \sim p^{\pi}(.|s,a)} [ r(s,a) + \gamma \mathcal{Q}^{\pi}(s', a') ]
\end{align*}
because $$\mathbb{E}_{p^{\pi}} [\sum_{t \geq 1} \gamma^{t-1} r(s_t, a_t) | s_1 = s', a_1 = a'] = \mathcal{Q}^{\pi}(s', a')$$
To prove the second equality, we split a policy $\pi$ into its immediate behaviour after $(s,a)$ and its subsequent behaviour: $\pi = (a_1, \pi')$. Using the before last line above
\begin{align*}
\mathcal{Q}^*(s,a) &= \max_{\pi} \mathcal{Q}^{\pi}(s,a) \\
&= \max_{\pi} [ r(s,a) + \gamma \sum_{(s', a')} p^{\pi}(s', a' | s, a) \mathcal{Q}^{\pi}(s', a') ] \\
&= r(s,a) + \gamma \max_{(a_1, \pi')} \sum_{(s', a')} p^{\pi}(s', a' | s, a) \mathcal{Q}^{\pi}(s', a') \\
&= r(s,a) + \gamma \max_{a_1} \max_{\pi'} \sum_{(s', a')} p^{a_1}(s', a' | s, a) \mathcal{Q}^{\pi'}(s', a') \\
&= r(s,a) + \gamma \max_{a_1} \sum_{(s', a')} p^{a_1}(s', a' | s, a) \max_{\pi'} \mathcal{Q}^{\pi'}(s', a') \\
&= r(s,a) + \gamma \max_{a_1} \sum_{(s', a')} p^{a_1}(s', a' | s, a) \mathcal{Q}^*(s', a') \\
&= r(s,a) + \gamma \max_{a_1} \sum_{(s', a')} p(s'| s, a) a_1(a' | s') \mathcal{Q}^*(s', a') \\
&= r(s,a) + \gamma \sum_{(s', a')} p(s'| s, a) \max_{a_1} a_1(a' | s') \mathcal{Q}^*(s', a') \\
&= r(s,a) + \gamma \sum_{s'} p(s' | s, a) \max_{a'}  \mathcal{Q}^*(s', a') \\
&= \mathbb{E}_{s' \sim p(.|s,a)}[r(s,a) + \gamma \max_{a'}  \mathcal{Q}^*(s', a')] 
\end{align*}
where we replaced the maximization over all one-step stochastic policies $a_1$ by a maximisation over all actions $a'$, as maximizing $a_1(a' | s') \mathcal{Q}^*(s', a')$ over $a_1$ will return the deterministic policy that maximizes $\mathcal{Q}^*(s', .)$
This loss represents the distance between the current value function and the imporovement of the value function by one-step look-ahead using the observed reward (the usual Q-learning step). The shape of the update is suggested by the Bellman equation for $\mathcal{Q}$ that we proved.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) == self.max_memory :
            del self.memory[0]
        self.memory.append(m) 
            

    def random_access(self):
        return self.memory[np.random.choice(np.arange(0, len(self.memory)))]


***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        q = self.model.predict(s.reshape(-1, 5, 5, self.n_state))
        return np.argmax(q)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            [s, n_s, a, r, game_over] = self.memory.random_access()
            input_states[i] = s
            target_q[i] = self.model.predict(s.reshape(-1, 5, 5, self.n_state))
            old_q = self.model.predict(n_s.reshape(-1, 5, 5, self.n_state))
            if game_over:
                target_q[i, a] = r
            else:
                target_q[i, a] = r + self.discount * np.max(old_q)
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__(*args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(4, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train60.mp4'))

Epoch 000/100 | Loss 0.0191 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/100 | Loss 0.0111 | Win/lose count 1.5/3.0 (-1.5)
Epoch 002/100 | Loss 0.0015 | Win/lose count 2.5/2.0 (0.5)
Epoch 003/100 | Loss 0.0039 | Win/lose count 1.0/3.0 (-2.0)
Epoch 004/100 | Loss 0.0054 | Win/lose count 4.5/1.0 (3.5)
Epoch 005/100 | Loss 0.0019 | Win/lose count 2.0/2.0 (0.0)
Epoch 006/100 | Loss 0.0074 | Win/lose count 3.5/3.0 (0.5)
Epoch 007/100 | Loss 0.0033 | Win/lose count 1.5/6.0 (-4.5)
Epoch 008/100 | Loss 0.0073 | Win/lose count 4.0/1.0 (3.0)
Epoch 009/100 | Loss 0.0038 | Win/lose count 2.0/2.0 (0.0)
Epoch 010/100 | Loss 0.0055 | Win/lose count 2.0/1.0 (1.0)
Epoch 011/100 | Loss 0.0069 | Win/lose count 4.5/7.0 (-2.5)
Epoch 012/100 | Loss 0.0558 | Win/lose count 4.0/1.0 (3.0)
Epoch 013/100 | Loss 0.0045 | Win/lose count 3.5/2.0 (1.5)
Epoch 014/100 | Loss 0.0016 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/100 | Loss 0.0418 | Win/lose count 1.5/4.0 (-2.5)
Epoch 016/100 | Loss 0.0054 | Win/lose count 4.0/6

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(64, (2, 2), input_shape = (5 , 5 , self.n_state), activation = 'relu'))
        model.add(Conv2D(32, (2, 2), activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train60.mp4'))

Epoch 000/100 | Loss 0.0038 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/100 | Loss 0.0028 | Win/lose count 4.0/1.0 (3.0)
Epoch 002/100 | Loss 0.0429 | Win/lose count 2.0/2.0 (0.0)
Epoch 003/100 | Loss 0.0011 | Win/lose count 3.0/5.0 (-2.0)
Epoch 004/100 | Loss 0.0022 | Win/lose count 7.0/4.0 (3.0)
Epoch 005/100 | Loss 0.0113 | Win/lose count 2.0/4.0 (-2.0)
Epoch 006/100 | Loss 0.0009 | Win/lose count 6.5/7.0 (-0.5)
Epoch 007/100 | Loss 0.0573 | Win/lose count 2.0/2.0 (0.0)
Epoch 008/100 | Loss 0.0032 | Win/lose count 4.5/3.0 (1.5)
Epoch 009/100 | Loss 0.0172 | Win/lose count 2.5/2.0 (0.5)
Epoch 010/100 | Loss 0.0081 | Win/lose count 2.5/3.0 (-0.5)
Epoch 011/100 | Loss 0.0644 | Win/lose count 3.5/3.0 (0.5)
Epoch 012/100 | Loss 0.0035 | Win/lose count 1.0/0 (1.0)
Epoch 013/100 | Loss 0.0030 | Win/lose count 2.0/2.0 (0.0)
Epoch 014/100 | Loss 0.0034 | Win/lose count 4.0/2.0 (2.0)
Epoch 015/100 | Loss 0.0042 | Win/lose count 2.0/0 (2.0)
Epoch 016/100 | Loss 0.0020 | Win/lose count 5.5/3.0 (2

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

for temp in [0.1, 0.3, 0.5, 0.7, 0.9] :
    print('\n------- Temperature:', temp)
    env = Environment(grid_size=size, max_time=T,temperature=temp)
    
    print('------- Test of the CNN')
    test(agent_cnn,env,epochs_test,prefix='cnn_test')
    print('------- Test of the FC')
    test(agent_fc,env,epochs_test,prefix='fc_test')


------- Temperature: 0.1
------- Test of the CNN
Win/lose count 7.0/0. Average score (7.0)
Win/lose count 2.0/0. Average score (4.5)
Win/lose count 3.0/0. Average score (4.0)
Win/lose count 2.0/0. Average score (3.5)
Win/lose count 5.5/0. Average score (3.9)
Win/lose count 3.0/0. Average score (3.75)
Win/lose count 10.5/0. Average score (4.714285714285714)
Win/lose count 6.5/0. Average score (4.9375)
Win/lose count 6.5/0. Average score (5.111111111111111)
Win/lose count 0.5/0. Average score (4.65)
Win/lose count 2.5/0. Average score (4.454545454545454)
Win/lose count 7.0/0. Average score (4.666666666666667)
Win/lose count 7.5/0. Average score (4.884615384615385)
Win/lose count 5.0/0. Average score (4.892857142857143)
Win/lose count 3.5/0. Average score (4.8)
Win/lose count 2.5/0. Average score (4.65625)
Win/lose count 6.5/0. Average score (4.764705882352941)
Win/lose count 6.0/0. Average score (4.833333333333333)
Win/lose count 5.0/0. Average score (4.842105263157895)
Win/lose count 5

Win/lose count 25.0/1.0. Average score (25.96875)
Win/lose count 23.5/0. Average score (25.823529411764707)
Win/lose count 36.0/0. Average score (26.38888888888889)
Win/lose count 36.0/0. Average score (26.894736842105264)
Win/lose count 33.0/0. Average score (27.2)
Final score: 27.2

------- Temperature: 0.9
------- Test of the CNN
Win/lose count 71.5/0. Average score (71.5)
Win/lose count 70.5/0. Average score (71.0)
Win/lose count 71.0/0. Average score (71.0)
Win/lose count 67.5/0. Average score (70.125)
Win/lose count 70.5/0. Average score (70.2)
Win/lose count 71.5/0. Average score (70.41666666666667)
Win/lose count 68.5/0. Average score (70.14285714285714)
Win/lose count 49.0/1.0. Average score (67.375)
Win/lose count 61.0/1.0. Average score (66.55555555555556)
Win/lose count 77.0/0. Average score (67.6)
Win/lose count 64.5/0. Average score (67.31818181818181)
Win/lose count 72.0/0. Average score (67.70833333333333)
Win/lose count 60.5/0. Average score (67.15384615384616)
Win/los

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

-  The scores get better for both algorithms as the temperature increases. A higher temperature means that there are more rewards to be collected, but also more poisonous cells (up to a certain temperature after which there is less poison). This shows that both algorithms learn to avoid the poisonous cells pretty well, which can also be seen in the videos.
-  The CNN always performs better than the FCNN.
-  We observe a lack of exploration when the temperature is low. If no cheese is in the vicinity of the rat, it does not explore other regions of the board.
-  Here is a table summarizing the average scores above for each algorithm as the temperature changes 

| Temperature | Average score of DQN CNN | Average score of DQN FC |
| --- | --- | --- |
| 0.1 | 4.85 | 1.9 |
| 0.3 | 13.75 | 6.35 |
| 0.5 | 24.175 | 12.725 |
| 0.7 | 43.8 | 27.2 |
| 0.9 | 63.975 | 53.575 |

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent, env, epoch, decay = 0.9, prefix = ''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon * decay)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) # malus if previously visited position

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board > 0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 # I changed this line. It was exactly the same as the line above it.

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train: # include malus
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1  # I changed this line. It was exactly the same as the line above it.
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.6, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0100 | Win/lose count 8.0/27.100000000000122 (-19.100000000000122)
Epoch 001/100 | Loss 0.0088 | Win/lose count 10.0/20.40000000000002 (-10.40000000000002)
Epoch 002/100 | Loss 0.0076 | Win/lose count 7.5/26.80000000000006 (-19.30000000000006)
Epoch 003/100 | Loss 0.0096 | Win/lose count 6.0/20.80000000000001 (-14.800000000000011)
Epoch 004/100 | Loss 0.0073 | Win/lose count 9.0/22.40000000000004 (-13.400000000000041)
Epoch 005/100 | Loss 0.0143 | Win/lose count 14.0/18.399999999999988 (-4.399999999999988)
Epoch 006/100 | Loss 0.0054 | Win/lose count 16.0/16.899999999999984 (-0.8999999999999844)
Epoch 007/100 | Loss 0.0089 | Win/lose count 7.0/15.69999999999996 (-8.69999999999996)
Epoch 008/100 | Loss 0.0085 | Win/lose count 14.0/18.099999999999987 (-4.099999999999987)
Epoch 009/100 | Loss 0.0068 | Win/lose count 4.5/20.40000000000001 (-15.90000000000001)
Epoch 010/100 | Loss 0.0050 | Win/lose count 24.0/11.699999999999974 (12.300000000000026)
Epoch 011/100 | Los

Epoch 092/100 | Loss 0.0180 | Win/lose count 24.0/11.799999999999976 (12.200000000000024)
Epoch 093/100 | Loss 0.0204 | Win/lose count 24.0/13.399999999999975 (10.600000000000025)
Epoch 094/100 | Loss 0.0190 | Win/lose count 24.5/8.199999999999987 (16.30000000000001)
Epoch 095/100 | Loss 0.0210 | Win/lose count 19.5/12.999999999999977 (6.500000000000023)
Epoch 096/100 | Loss 0.0210 | Win/lose count 22.0/11.299999999999981 (10.700000000000019)
Epoch 097/100 | Loss 0.0223 | Win/lose count 25.0/12.999999999999973 (12.000000000000027)
Epoch 098/100 | Loss 0.0314 | Win/lose count 20.0/8.999999999999984 (11.000000000000016)
Epoch 099/100 | Loss 0.0298 | Win/lose count 24.5/11.099999999999978 (13.400000000000022)


In [20]:
# Evaluation
test(agent, env, epochs_test, prefix = 'cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 17.5/4.0. Average score (13.5)
Win/lose count 25.5/6.0. Average score (16.5)
Win/lose count 28.0/1.0. Average score (20.0)
Win/lose count 25.0/6.0. Average score (19.75)
Win/lose count 27.0/0. Average score (21.2)
Win/lose count 24.5/3.0. Average score (21.25)
Win/lose count 23.5/0. Average score (21.571428571428573)
Win/lose count 21.5/4.0. Average score (21.0625)
Win/lose count 19.0/0. Average score (20.833333333333332)
Win/lose count 25.5/2.0. Average score (21.1)
Win/lose count 28.0/1.0. Average score (21.636363636363637)
Win/lose count 33.0/0. Average score (22.583333333333332)
Win/lose count 28.5/1.0. Average score (22.96153846153846)
Win/lose count 23.0/0. Average score (22.964285714285715)
Win/lose count 23.0/0. Average score (22.966666666666665)
Win/lose count 26.0/1.0. Average score (23.09375)
Win/lose count 12.5/0. Average score (22.470588235294116)
Win/lose count 31.0/0. Average score (22.944444444444443)
Win/lose count 21.0/0. Average score (22.8421052631578

-  We notice a large improvement in performance (~22 vs ~13 at temperature 0.3) and a good explorative behaviour in the video.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

-  I trained a CNN-DQN that mimicks the behaviour of an expert agent by assigning a high positive value to the action taken by the expert and the value 0 to all other actions. This agent trains while interacting with the environment in the following manner: when met with state $s$, it asks the expert for its chosen action $a$ and then calls ```reinforce``` on $(s,a)$. The reward and the next state are not needed as the agent simply trusts the choice of the expert. Its ```reinforce``` function learns from a sampled minibatch of $(s,a)$ decisions made by the expert by constructing a target ```target_q``` containing a high value for action $a$ in state $s$ and zero for the other actions. 
-  Below we can see that this agent performs worse than the agent it mimicks (~18 at temperature 0.3), but still better than the agent in questions 8 and 9 before encouraged exploration (~14 at temperature 0.3). We also notice that this agent is quick to train.

In [21]:
class DQN_mimick(DQN):
    def __init__(self, *args,**kwargs):
        super(DQN_mimick, self).__init__(*args,**kwargs)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, a_]) # only state and action needed
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        for i in range(self.batch_size):
            [s, a] = self.memory.random_access()
            input_states[i] = s
            target_q[i,a] = 10  # high value to the action we want to mimick, zero for other actions
            
        l = self.model.train_on_batch(input_states, target_q)
        return l
    
class DQN_mimick_CNN(DQN_mimick):
    def __init__(self, *args, lr = 0.1, **kwargs):
        super(DQN_mimick_CNN, self).__init__(*args, **kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, (2, 2), input_shape = (5 , 5 , self.n_state), activation = 'relu'))
        model.add(Conv2D(32, (2, 2), activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr = lr, decay = 1e-4, momentum = 0.0), "mse")
        self.model = model

In [22]:
def train_mimick(agent, expert, env , epoch, prefix = ''):
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # Get the expert's action
            action = expert.act(state, train = False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [23]:
# Training
expert = agent
mimick_agent = DQN_mimick_CNN(size,lr=.1,epsilon=0.1,memory_size=2000,batch_size=32,n_state=3)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimick(mimick_agent, expert, env, 200, prefix = 'cnn_train_mimick')
HTML(display_videos('cnn_train_mimick60.mp4'))

Epoch 000/200 | Loss 6.4464 | Win/lose count 21.5/1.0 (20.5)
Epoch 001/200 | Loss 6.3933 | Win/lose count 28.5/1.0 (27.5)
Epoch 002/200 | Loss 3.5742 | Win/lose count 25.5/1.0 (24.5)
Epoch 003/200 | Loss 6.8083 | Win/lose count 22.0/0 (22.0)
Epoch 004/200 | Loss 8.8816 | Win/lose count 24.0/2.0 (22.0)
Epoch 005/200 | Loss 4.8010 | Win/lose count 20.0/0 (20.0)
Epoch 006/200 | Loss 7.6264 | Win/lose count 23.5/3.0 (20.5)
Epoch 007/200 | Loss 5.8832 | Win/lose count 20.0/2.0 (18.0)
Epoch 008/200 | Loss 3.7340 | Win/lose count 21.5/0 (21.5)
Epoch 009/200 | Loss 5.7990 | Win/lose count 9.0/0 (9.0)
Epoch 010/200 | Loss 6.6988 | Win/lose count 25.0/0 (25.0)
Epoch 011/200 | Loss 6.0037 | Win/lose count 21.5/0 (21.5)
Epoch 012/200 | Loss 5.7822 | Win/lose count 28.0/0 (28.0)
Epoch 013/200 | Loss 6.8454 | Win/lose count 27.0/4.0 (23.0)
Epoch 014/200 | Loss 5.4746 | Win/lose count 19.5/1.0 (18.5)
Epoch 015/200 | Loss 3.8849 | Win/lose count 20.5/0 (20.5)
Epoch 016/200 | Loss 4.8658 | Win/lose cou

Epoch 137/200 | Loss 5.9739 | Win/lose count 22.0/4.0 (18.0)
Epoch 138/200 | Loss 3.9335 | Win/lose count 26.0/1.0 (25.0)
Epoch 139/200 | Loss 2.9419 | Win/lose count 17.0/0 (17.0)
Epoch 140/200 | Loss 5.9151 | Win/lose count 24.0/1.0 (23.0)
Epoch 141/200 | Loss 7.0249 | Win/lose count 25.0/1.0 (24.0)
Epoch 142/200 | Loss 5.1611 | Win/lose count 18.5/1.0 (17.5)
Epoch 143/200 | Loss 3.8498 | Win/lose count 9.5/0 (9.5)
Epoch 144/200 | Loss 7.6779 | Win/lose count 28.5/0 (28.5)
Epoch 145/200 | Loss 6.6622 | Win/lose count 18.5/1.0 (17.5)
Epoch 146/200 | Loss 5.3193 | Win/lose count 16.0/1.0 (15.0)
Epoch 147/200 | Loss 4.7573 | Win/lose count 20.0/0 (20.0)
Epoch 148/200 | Loss 3.2835 | Win/lose count 25.5/3.0 (22.5)
Epoch 149/200 | Loss 6.2069 | Win/lose count 21.0/0 (21.0)
Epoch 150/200 | Loss 2.1074 | Win/lose count 11.0/0 (11.0)
Epoch 151/200 | Loss 5.7556 | Win/lose count 24.0/1.0 (23.0)
Epoch 152/200 | Loss 5.1288 | Win/lose count 19.5/0 (19.5)
Epoch 153/200 | Loss 7.7234 | Win/lose c

In [24]:
# Evaluation
test(mimick_agent, env, epochs_test, prefix = 'cnn_test_mimick')
HTML(display_videos('cnn_test_mimick10.mp4'))

Win/lose count 22.5/1.0. Average score (21.5)
Win/lose count 14.0/2.0. Average score (16.75)
Win/lose count 24.5/4.0. Average score (18.0)
Win/lose count 11.0/4.0. Average score (15.25)
Win/lose count 26.0/2.0. Average score (17.0)
Win/lose count 25.5/0. Average score (18.416666666666668)
Win/lose count 25.5/2.0. Average score (19.142857142857142)
Win/lose count 22.5/3.0. Average score (19.1875)
Win/lose count 20.5/4.0. Average score (18.88888888888889)
Win/lose count 23.5/0. Average score (19.35)
Win/lose count 16.0/2.0. Average score (18.863636363636363)
Win/lose count 18.0/1.0. Average score (18.708333333333332)
Win/lose count 24.5/1.0. Average score (19.076923076923077)
Win/lose count 21.0/2.0. Average score (19.071428571428573)
Win/lose count 9.5/1.0. Average score (18.366666666666667)
Win/lose count 21.0/0. Average score (18.53125)
Win/lose count 23.5/3.0. Average score (18.647058823529413)
Win/lose count 21.0/1.0. Average score (18.72222222222222)
Win/lose count 13.5/0. Average 

***